In [4]:
#!pip install jiwer

In [6]:
from jiwer import wer
import random
import pandas as pd

In [14]:
df = pd.read_csv("C://Users//garci//OneDrive//Área de Trabalho//Ambiente de trabalho unbunto//Cienciadedados//IPL//Tese//com_transcricao_simulada.csv")
df["wer"] = df.apply(lambda row: wer(str(row["referencia"]), str(row["transcricao_simulada"])), axis=1)
print(df[["referencia", "transcricao_simulada", "wer"]].head())
media_wer = df["wer"].mean()
print(f"\nWER médio: {media_wer:.2%}")


                                          referencia  \
0  "Boa tarde, senhor João pereira. Gostaria de c...   
1                                             "Sim."   
2  "Ótimo. Vamos conversar sobre um incidente oco...   
3  "Antes de tudo, pode me contar onde o senhor e...   
4  Encontrava-me nas Compras no Continente do Col...   

                                transcricao_simulada       wer  
0  "Boa tarde, senhor João pereira. Gostaria de c...  0.135135  
1                                             "Sim."  0.000000  
2  "Ótimo. Vamos sobre um incidente ocorrido rece...  0.157895  
3  "Antes de tudo, pode me contar onde o senhor e...  0.142857  
4  Encontrava-me nas Compras no Continente do Col...  0.090909  

WER médio: 15.93%
